# Imports

In [ ]:
# initial setup
import os
import asf_search as asf
from eof.download import download_eofs
from shapely.geometry import Polygon
import pandas as pd
import geopandas as gpd
import cartopy
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# Settings

In [ ]:
# paths
data_dir = 'data'
download_folder = os.path.join(data_dir,'scenes')
credentials_path = "credentials/credentials_earthdata.txt"
precise_orb_download_folder = os.path.join(data_dir,'orbits','POEORB')
restituted_orb_download_folder = os.path.join(data_dir,'orbits','RESORB')
dem_download_folder = os.path.join(data_dir,'dem')

download = True # whether to download files

In [ ]:
create_folders = True
if create_folders:
    for f in [download_folder, precise_orb_download_folder, restituted_orb_download_folder]:
        os.makedirs(f, exist_ok=True)

# Credentials

In [ ]:
# read credentials from file
with open(credentials_path, "r") as f:
   txt = str(f.read())
   uid = txt.split('\n')[1].split('login')[-1][1:]
   pswd = txt.split('\n')[2].split('password')[-1][1:]
   email = txt.split('\n')[3].split('email')[-1][1:]

# AOI

In [ ]:
#wkt = "POINT (113.66396 -66.163448)" # Antarctica - Law Dome 
#wkt = "POINT (166.7541 -77.7018)" # Antarctica - Ross Island (CHECK)
#wkt = "POINT (76.19525 -69.4068)" # Antarctica - Bharati/Davis Station
#wkt = "POINT (11.7308 -70.76683)" # Antarctica - Maitri Station
# wkt = "POINT (73.543 -53.1057)" # Antarctica - Heard/McDonald Island 

# wkt = "POINT (150.5759  -27.0252)" # Austraia - Surat Basin CRs 
# wkt = "POINT (115.3461 -29.0474)" # Austraia - Yaragadee 
# wkt = "POINT (147.533333 -25.533333)" # Austraia - Queensland Forest (-25o32'S and 147o32'E)

wkt = 'POLYGON ((179 -80, 179.9 -80, 179.9 -65, 179 -65, 179 -80))' # Antarctica - Antimeridian, (bl,br,ur,ul,br)
wkt = 'POLYGON ((179 -80, 179.9 -80, 179.9 -75, 179 -75, 179 -80))' # Antarctica - Antimeridian, (bl,br,ur,ul,br)

print(f'Searching over point: {wkt}')

# Search

### General time and area search

In [ ]:
# product type https://github.com/asfadmin/Discovery-asf_search/blob/master/asf_search/constants/PRODUCT_TYPE.py
# prod = 'GRD_HS' # IW
# prod = 'GRD_MD' # EW
prod = 'SLC' # IW, SM

# prod = 'GRD_HD'
# prod = 'GRD_MS'
# prod = 'GRD_FD'

# acquisition mode
mode = 'IW'
#mode = 'EW'
#mode = ['S1','S2','S3','S4','S5','S6'] # SM

start_date = '2014-01-01T00:00:00Z'
end_date = '2024-02-28T00:00:00Z'

results = asf.search(platform=[asf.PLATFORM.SENTINEL1], 
                     intersectsWith=wkt, 
                     maxResults=1000, 
                     processingLevel=prod,
                     beamMode=mode,
                     start=start_date,
                     end=end_date)
print(len(results))

### OR - Search for known products by their name

In [ ]:
search_list = True
if search_list:

    prod = 'SLC'
    mode = 'IW'
    
    scene_list = [
        # Antarctica - Maitri Station (-70.76683, 11.7308)
        'S1B_IW_SLC__1SSH_20190327T195016_20190327T195045_015544_01D236_9504',
        # 'S1B_IW_SLC__1SSH_20190526T195018_20190526T195048_016419_01EE8D_53BC',
        # 'S1B_IW_SLC__1SSH_20190315T195015_20190315T195045_015369_01CC73_DB8B',
        # # Antarctica - Bharati Station (-69.4068, 76.19525)
        # 'S1B_IW_SLC__1SSH_20190223T222639_20190223T222706_015079_01C2E9_1D63',
        # 'S1A_IW_SLC__1SSH_20190605T222724_20190605T222751_027550_031BE1_AD3A',
        # # Antarctica - Law Dome (-66.163448 113.66396)
        # 'S1B_IW_SLC__1SSH_20211204T123606_20211204T123635_029875_039105_E7EC',
        # 'S1A_IW_SLC__1SSH_20211210T123647_20211210T123716_040946_04DCF0_9FBA',
        # 'S1B_IW_SLC__1SSH_20211216T123605_20211216T123634_030050_03968D_9501',
        # # Antarctica - Ross Island (-77.7018, 166.7541)
        # 'S1A_IW_SLC__1SSH_20211208T124745_20211208T124815_040917_04DBED_5CFF',
        # 'S1A_IW_SLC__1SSH_20211220T124745_20211220T124815_041092_04E1C2_0475',
        # 'S1A_IW_SLC__1SSH_20220101T124744_20220101T124814_041267_04E7A2_1DAD',
        # # Antarctica - Heard/McDonald Island (-53.1057, 73.543) 
        # 'S1A_IW_SLC__1SSH_20220103T141935_20220103T142005_041297_04E8B4_9B44',
        # 'S1A_IW_SLC__1SSH_20220108T142747_20220108T142815_041370_04EB2C_4E64',
        # 'S1A_IW_SLC__1SSH_20220115T141935_20220115T142005_041472_04EE7C_9E3D',
        # # Antarctica - Antimeridian (-70, 180) approx
        # 'S1A_IW_SLC__1SSH_20220203T153614_20220203T153644_041750_04F7DF_8C3E',
        # 'S1A_IW_SLC__1SSH_20220122T153615_20220122T153645_041575_04F1EF_9254',
        # # Antarctica - Antimeridian (-75, 180) approx
        # 'S1A_IW_SLC__1SDV_20191028T131928_20191028T131947_029659_0360CA_A1A0',
        # 'S1A_IW_SLC__1SDH_20161201T131103_20161201T131131_014186_016EB7_F766',
        # # Australia - Yaragadee (-29.0474, 115.3461) Descending (Only Avail)
        # 'S1B_IW_SLC__1SDV_20211201T214143_20211201T214210_029837_038FD8_81C3',
        # 'S1B_IW_SLC__1SDV_20211201T214208_20211201T214240_029837_038FD8_8B37',
        # 'S1B_IW_SLC__1SDV_20211213T214143_20211213T214210_030012_039555_8753',
        # # Australia - Queensland Forest (-25.533333 147.533333)
        # 'S1A_IW_SLC__1SDV_20220121T193807_20220121T193834_041563_04F181_5804',
        # 'S1A_IW_SLC__1SDV_20220202T084138_20220202T084206_041731_04F740_197D',
        # 'S1A_IW_SLC__1SDV_20220202T084204_20220202T084231_041731_04F740_400F',
        # # Australia - SURAT QLD corner reflector (-27.0252,150.5759) - ASCENDING MODE (REQUIRED)
        # 'S1A_IW_SLC__1SSH_20220116T083314_20220116T083342_041483_04EED2_DB08',
        # 'S1A_IW_SLC__1SSH_20220104T083315_20220104T083342_041308_04E91B_8E5F',
        # 'S1B_IW_SLC__1SSH_20211216T123605_20211216T123634_030050_03968D_9501',
   ]

    # search the scene list with the specified product type
    results = asf.granule_search(scene_list, asf.ASFSearchOptions(processingLevel=prod))

In [ ]:
location_data = {'scene' : [], 'geometry' : []}

for r in results:
      scene_id = r.properties['sceneName']
      print(scene_id)
      points = (r.__dict__['umm']['SpatialExtent']['HorizontalSpatialDomain']
                  ['Geometry']['GPolygons'][0]['Boundary']['Points'])
      points = [(p['Longitude'],p['Latitude']) for p in points]
      scene_poly = Polygon(points)
      location_data['geometry'].append(scene_poly)
      location_data['scene'].append(scene_id)

In [ ]:
df = pd.DataFrame.from_dict(location_data)
df = gpd.GeoDataFrame(df, geometry='geometry',crs=4326)
df = df.to_crs(3031)
df.plot()

In [ ]:
def plot_results_footprint_map(df, 
                               title='', 
                               group='', 
                               group_colors = {},
                               sort='', 
                               legend_title='', 
                               crs=3031,
                               bounds=(-180, 180, -90, -50)):
    # plot the the product geometries on a map
    east, west, south, north = bounds
    plt.rcParams["figure.figsize"] = [10,8]
    ax = plt.axes(projection=cartopy.crs.SouthPolarStereo())
    ax.set_extent((east, west, south, north+1), cartopy.crs.PlateCarree())
    ax.add_feature(cartopy.feature.LAND)
    ax.add_feature(cartopy.feature.OCEAN)
    proj = cartopy.crs.SouthPolarStereo() if crs==3031 else cartopy.crs.PlateCarree()
    if not group:
        ax.add_geometries(df.geometry, crs=proj, alpha=0.3, edgecolor='black')
    else:
        groups = df[group].unique() if not sort else df.sort_values(sort)[group].unique()
        # generate N visually distinct colours
        colors = plt.rcParams['axes.prop_cycle'].by_key()['color']
        legend = []
        for i,g in enumerate(groups):
           colour = colors[i] if not group_colors else group_colors[g]
           ax.add_geometries(df[df[group]==g].geometry, crs=proj, alpha=0.3, color=colour) 
           legend.append(mpatches.Patch(color=colour, label=g))
        ax.legend(handles=legend, title= group if not legend_title else legend_title)
    ax.gridlines(draw_labels=True)
    ax.add_feature(cartopy.feature.COASTLINE)
    plt.title(title)
    plt.show()

In [ ]:
df.to_file('test_sites_3031.json')

In [ ]:
plot_results_footprint_map(df, 
                            title='', 
                            group='', 
                            group_colors = {},
                            sort='', 
                            legend_title='', 
                            crs=3031,
                            bounds=(-180, 180, -90, -50))

# Download Scenes

In [ ]:
# download
session = asf.ASFSession()
session.auth_with_creds(uid,pswd)

In [ ]:
# download all results
scene_paths = []
scene_names = []
if download:
    for scene in results:
        name = scene.__dict__['umm']['GranuleUR'].split('-')[0]
        scene_names.append(name)
        print(name)
        path = os.path.join(download_folder, name)
        scene.download(path=download_folder, session=session)
        scene_paths.append(path)
        break

# Download Precise Orbit Files

In [ ]:
for scene in  scene_paths:
    print(scene)
    # download precise orbits
    download_eofs(sentinel_file=scene, save_dir=precise_orb_download_folder, orbit_type='precise')
    # download restituted orbits
    download_eofs(sentinel_file=scene, save_dir=restituted_orb_download_folder, orbit_type='restituted')

In [ ]:
print('precise orbits')
for p in os.listdir(precise_orb_download_folder):
    print(os.path.join(precise_orb_download_folder,p))

print('restited orbits')
for p in os.listdir(restituted_orb_download_folder):
    print(os.path.join(restituted_orb_download_folder,p))

# DEMs

In [ ]:
# add your credentials to a .netrc file
# https://pypi.org/project/dem-stitcher/

In [ ]:
import rasterio
from dem_stitcher import stitch_dem
from shapely.geometry import Polygon

# valid DEM list here - # https://pypi.org/project/dem-stitcher/
dem_name = 'glo_30'  # Global Copernicus 30 meter resolution DEM

for r in results:
    scene_name = r.__dict__['umm']['GranuleUR'].split('-')[0]
    print(r.__dict__['umm']['GranuleUR'].split('-')[0])
    points = (r.__dict__['umm']['SpatialExtent']['HorizontalSpatialDomain']
              ['Geometry']['GPolygons'][0]['Boundary']['Points'])
    points = [(p['Longitude'],p['Latitude']) for p in points]
    poly = Polygon(points)
    #bounds = poly.bounds # as xmin, ymin, xmax, ymax in epsg:4326
    bounds = poly.buffer(1).bounds #buffered
    print(f'downloding DEM for scene bounds : {bounds}')
    # poly_3031 = transform_polygon(4326, 3031, poly, always_xy=True)
    # poly = antimeridian.fix_polygon(poly)
   
    # get the DEM and geometry information
    X, p = stitch_dem(bounds,
                    dem_name=dem_name,
                    dst_ellipsoidal_height=False,
                    dst_area_or_point='Point')

    # make folders and set filenames
    dem_dl_folder = os.path.join(dem_download_folder,dem_name)
    os.makedirs(dem_dl_folder, exist_ok=True)
    dem_filename = scene_name + '_dem.tif'
    save_path = os.path.join(dem_dl_folder,dem_filename)
    
    # save with rasterio
    print(f'saving dem to {save_path}')
    with rasterio.open(save_path, 'w', **p) as ds:
        ds.write(X, 1)
        ds.update_tags(AREA_OR_POINT='Point')